Previous R note shows you:

1. how to do ESDA with rGeoDa and sp by computing the local spatial statistics (LISA) of variable hr60 and creating a LISA cluster map.

Another modern and popular routine for spatial data analysis with R is to use sf library to process geospatial dataset, analyze spatial data and plot the results on maps.

rgeoda also provides utility functions to create a rgeoda instance from sf object.

In this note, I will demonstrate how to work with sf and rgeoda packages to do exploratory spatial data analysis.

# 1. Start from sf package

The sf package has been popular tool to handle geospatial data. It is a good substitue of sp package which will be deprecated soon.

One can use sf to load geospatial data (e.g. a ESRI Shapefile) directly:

In [ ]:
library(rgeoda)
guerry_path <- system.file("extdata", "Guerry.shp", package = "rgeoda")

In [ ]:
library(sf)

Using `st_read()` function to read Guerry.shp file and this function will returns an `sf` object.

In [ ]:
guerry_sf <- st_read(guerry_path)

You can simply call plot() function to render the first 9 chorepleth maps using the frist 9 variables in the dataset:



In [ ]:
plot(guerry_sf)

# 2. Create rgeoda object from sf

sf package makes it easy to work with geospatial data. It also provides functions to do basic spatial data analysis. rgeoda provides helper function `sf_to_geoda` to create a GeoDa instance from a sf object. Users can then call GeoDa functions to do further spatial data analysis.



In [ ]:
guerry_gda <- sf_to_geoda(guerry_sf)


rgeoda uses wkb, which is a binary representation of geometries, to exchange data between sf and libgeoda in memory.

# 3. ESDA with rgeoda

Now, with the rgeoda object `guerry_gda`, you can call GeoDa’s spatial analysis functions. For example, to examine the local Moran of variable “crm_prs” (Population per Crime against persons):



In [ ]:
queen_w <- guerry_gda$CreateContiguityWeights()
crm_lisa <- guerry_gda$LISA(queen_w, as.numeric(paste(guerry_sf$Crm_prs)))
crm_lisa_clusters <- crm_lisa$GetClusterIndicators()

Now, with the LISA results, we can do exploratory spatial data analysis by generating a LISA cluster map:



In [ ]:
colors <- sapply(crm_lisa_clusters, function(x){return(lisa_colors[[x+1]])})
plot(st_geometry(guerry_sf), col = colors, border = "#333333", lwd=0.2)
title(main = "LISA of crm_prs")
legend('bottomright', legend = lisa_labels, 
       fill = lisa_colors, border = "#eeeeee")

From the above code, you can see that we still use `sf` object to do plotting. The values of cluster indicators from rgeoda’s LISA object are used to make the LISA map.

You can easily append the lisa results to original sf object by manipulating the data.frame object inside the sf object.

We then create an array of hex color, each observation is set to a hex color according to its cluster indicator value.

In [ ]:
sapply(crm_lisa_clusters, function(x){return(lisa_colors[[x+1]])})


If you check the values of the cluster indicators, you will see they are integer numbers 0 (not significant), 1 (high-high cluster), 2 (low-low cluster), 3 (low-high cluster), 4 (high-low cluster), 5 (neighborless/island), 6 (undefined), which are excatly the same with GeoDa software when you save LISA results to a table:

In [ ]:
crm_lisa_clusters

We also specify what color is associated with which cluster value to replace the lisa_colors and lisa_labels, which are defined in sf_geoda.R:

'''
lisa_colors <- c("#eeeeee","#FF0000","#0000FF","#a7adf9", "#f4ada8", "#999999")
'''